# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 9:10AM,239404,18,CLRSPEC-1111182,"ClearSpec, LLC",Released
1,Jun 13 2022 9:07AM,239397,10,0085755303,ISDIN Corporation,Released
2,Jun 13 2022 9:07AM,239397,10,0085755307,ISDIN Corporation,Released
3,Jun 13 2022 9:07AM,239397,10,0085755308,ISDIN Corporation,Released
4,Jun 13 2022 9:07AM,239397,10,0085755309,ISDIN Corporation,Released
5,Jun 13 2022 9:07AM,239397,10,0085755311,ISDIN Corporation,Released
6,Jun 13 2022 9:07AM,239397,10,0085755310,ISDIN Corporation,Released
7,Jun 13 2022 9:07AM,239397,10,0085755316,ISDIN Corporation,Released
8,Jun 13 2022 9:07AM,239397,10,0085755317,ISDIN Corporation,Released
9,Jun 13 2022 9:07AM,239397,10,0085755318,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,239397,Released,50
16,239398,Released,14
17,239399,Released,3
18,239402,Released,1
19,239404,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
239397,NaN,50.0
239398,NaN,14.0
239399,NaN,3.0
239402,NaN,1.0
239404,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
239367,1.0,1.0
239386,18.0,12.0
239387,15.0,6.0
239388,0.0,1.0
239389,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
239367,1,1
239386,18,12
239387,15,6
239388,0,1
239389,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,239367,1,1
1,239386,18,12
2,239387,15,6
3,239388,0,1
4,239389,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,239367,1,1
1,239386,18,12
2,239387,15,6
3,239388,,1
4,239389,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC"
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation
51,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation
101,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.
115,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC"
116,Jun 13 2022 9:03AM,239399,10,"Snap Medical Industries, LLC"
119,Jun 13 2022 8:56AM,239396,18,"ClearSpec, LLC"
120,Jun 13 2022 8:55AM,239394,10,"Amcyte Pharma, Inc."
123,Jun 13 2022 8:52AM,239393,10,Beach Products Inc
137,Jun 13 2022 8:47AM,239392,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",,1
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,,50
2,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,,50
3,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.,,14
4,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC",,1
5,Jun 13 2022 9:03AM,239399,10,"Snap Medical Industries, LLC",,3
6,Jun 13 2022 8:56AM,239396,18,"ClearSpec, LLC",,1
7,Jun 13 2022 8:55AM,239394,10,"Amcyte Pharma, Inc.",,3
8,Jun 13 2022 8:52AM,239393,10,Beach Products Inc,,14
9,Jun 13 2022 8:47AM,239392,18,"ClearSpec, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",1,
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,50,
2,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,50,
3,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.,14,
4,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC",1,
5,Jun 13 2022 9:03AM,239399,10,"Snap Medical Industries, LLC",3,
6,Jun 13 2022 8:56AM,239396,18,"ClearSpec, LLC",1,
7,Jun 13 2022 8:55AM,239394,10,"Amcyte Pharma, Inc.",3,
8,Jun 13 2022 8:52AM,239393,10,Beach Products Inc,14,
9,Jun 13 2022 8:47AM,239392,18,"ClearSpec, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",1,
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,50,
2,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,50,
3,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.,14,
4,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",1,NaN
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,50,NaN
2,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,50,NaN
3,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.,14,NaN
4,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",1,0.0
1,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,50,0.0
2,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,50,0.0
3,Jun 13 2022 9:06AM,239398,20,Alumier Labs Inc.,14,0.0
4,Jun 13 2022 9:04AM,239402,18,"ClearSpec, LLC",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1915126,138,1.0
18,957594,4,0.0
19,478773,18,33.0
20,239398,14,0.0
22,478777,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1915126,138,1.0
1,18,957594,4,0.0
2,19,478773,18,33.0
3,20,239398,14,0.0
4,22,478777,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,138,1.0
1,18,4,0.0
2,19,18,33.0
3,20,14,0.0
4,22,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,18,Released,4.0
2,19,Released,18.0
3,20,Released,14.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,18,19,20,22
Status,,,,,
Executing,1.0,0.0,33.0,0.0,0.0
Released,138.0,4.0,18.0,14.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,18,19,20,22
0,Executing,1.0,0.0,33.0,0.0,0.0
1,Released,138.0,4.0,18.0,14.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,18,19,20,22
0,Executing,1.0,0.0,33.0,0.0,0.0
1,Released,138.0,4.0,18.0,14.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()